<p style="color:Red"><font size="5">Stochastic Diffusion Search</font></p>

<p ><font size="3">SDS is an agent-based algorithm that can be used in a variety of contexts to solve different problems. We will be using a flavor of SDS to solve the problem of "curse of dimensionality" in the datasets. I recommend you to read this intuitive <a href="http://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification">article</a> on curse of  dimensionality and its repercussions</font></p>

<p ><font size="3"> The theory behind the algorithm that I coded below is originally published <a href="https://dl.acm.org/citation.cfm?id=3079193">here</a>. I strongle recommend to give a read before trying the algorithm. However, I made some key improvements to the original algorithm</font></p>

In [1]:
#importing necessary libraries
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from pprint import pprint

In [2]:
#Creating instances of estimators
logReg=LogisticRegression(C=2)
decClf=DecisionTreeClassifier(max_depth=5, min_samples_split=4)
svc=SVC(C=0.5,gamma=0.5)

In [3]:
estimators=[svc,logReg,decClf] 

In [4]:
'''
Below function returns an agent, which is hypothesis, and its corresponding binary array.
1 indicates inclusion of corresponding feature and 0 indicates exclusion of the feature.
lowerLim indicates minimum number of features, whereas; upperLim indicated max no of features to beincluded in an agent.
'''
def agent(arryX,lowerLim,upperLim):
        if lowerLim<0 or upperLim>arryX.shape[1]:
            print('recall function with appropriate limits')
        else:
            randomNoFeatures=np.random.randint(lowerLim,upperLim,size=1)[0] #generating a random number
            zeroArry=np.zeros(arryX.shape[1]-randomNoFeatures, dtype='int') #zero array 
            oneArry=np.ones(randomNoFeatures, dtype='int')   #one array
            fArry=np.concatenate((zeroArry,oneArry), axis=0) #concatinating zero and one array
            np.random.shuffle(fArry) #shuffling fArray
            fIndex=np.where(fArry==1)[0]
            agentArry=arryX[:,fIndex] #generating feature subset from origanal dataset
            return fArry,agentArry

In [5]:
'''
Below function generates required number of agents that are to be deployed on search space. 
All the agents and corresponding binary feature array are stored and returned as a list.
'''
def agentsInitiation(arryX,numAgents,lowerLim,upperLim):
        agents=[]
        agentFIndex=[]
        agentStatus=['active']*numAgents
        for i in range(0,numAgents):
            fArry,agentArry=agent(arryX,lowerLim,upperLim) #generating a single agent
            agentFIndex.append(fArry) #appending its binary feature array to agentFIndex
            agents.append(agentArry) #appending the agent to the agents list
        return agents,agentFIndex,agentStatus

In [6]:
'''
'Score' function fits each model to the agent's training data and then evaluates the score on test agent. 
The output is the average score of three estimators. Original paper used only one classifier to calculate score.
Therefore, the resultant subset was very biased towards the signle estimator. 
To avoind this, we are using ensemble of classifiers.
'''
def score(estimators,arryX,arrY):
        X_train,X_test,y_train,y_test=train_test_split(arryX,arrY,random_state=0)
        scores=[]
        for i in range(len(estimators)):
            estimators[i].fit(X_train,y_train) #fitting the ith estimator to the training data of an agent
            scores.append(estimators[i].score(X_test,y_test)) #evaluating the score on the test data
        return sum(scores)/len(scores)

In [7]:
#below function calculates score for each agents and appends the score to the agentScores list
def agentClfscores(estimators,agents,arrY):
    agentScores=[]
    for agent in agents:
        agentscore=score(estimators,agent,arrY)
        agentScores.append(agentScores)
    return agentScores #returns a list that caputres agents' scores

In [29]:
'''
Below function carries out test and diffusion phase among the agents initialized above. 
The function returns agents and their corresponding scores, binary feature arrays, and staus after numIterations.
'''

def SDSFS(arryX,arrY,estimators,numIterations,numAgents,lowerLim,upperLim):
    agents,agentFIndex,agentStatus=agentsInitiation(arryX,numAgents,lowerLim,upperLim)
    agentScores=agentClfscores(estimators,agents,arrY)
    niters=0
    while niters<numIterations:
        #testing phase
        for i in range(len(agents)):
            rndmId=np.random.randint(len(agents),size=1)[0]
            if agentScores[i]>agentScores[rndmId]:
                agentStatus[i]='active' 
                
            else:
                agentStatus[i]='inactive'
                
        #Diffusion phase    
        for i in range(len(agents)):
                if agentStatus[i]=='inactive':
                    rndmId2=np.random.randint(len(agents),size=1)[0]
                    if agentStatus[rndmId2]=='active':
                        oneIds=np.where(agentFIndex[rndmId2]==1)[0]
                        zeroIds=np.where(agentFIndex[rndmId2]==0)[0]
                        rndmId3=np.random.randint(len(oneIds), size=1)
                        rndmId4=np.random.randint(len(zeroIds), size=1)
                        oneZeroId=oneIds[rndmId3]
                        zeroOneId=zeroIds[rndmId4]
                        agentFIndex[i]=agentFIndex[rndmId2].copy()
                        agentFIndex[i][oneZeroId]=0
                        agentFIndex[i][zeroOneId]=1
                        fIndex2=np.where(agentFIndex[i]==1)[0]
                        agents[i]=X[:,fIndex2]
                        agentScores[i]=score(estimators,agents[i],arrY)
                    else:
                        agentFIndex[i],agents[i]=agent(arryX,lowerLim,upperLim)
                        agentScores[i]=score(estimators,agents[i],arrY)
                else:
                    rndmId5=np.random.randint(len(agents),size=1)[0]
                    if agentStatus[rndmId5]=='active' and (agentFIndex[i]==agentFIndex[rndmId5]).all():
                        agentStatus[i]='inactive'
                        agentFIndex[i],agents[i]=agent(arryX,lowerLim,upperLim)
                        agentScores[i]=score(estimators,agents[i],arrY)
        niters+=1
    return agents,agentFIndex,agentStatus,agentScores
    

<p style="color:Red"><font size="5">Testing SDSFS on a dataset</font></p>

The dataset contains 111 patterns obtained by bouncing sonar signals off a metal cylinder at various angles and under various conditions. 
It contains 97 patterns obtained from rocks under similar conditions. 
The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. 
The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock. 

In [10]:
df=pd.read_csv('files/Sonar.csv')

In [11]:
df.shape #the dataset has 60 features and one class column

(208, 61)

In [12]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1


In [13]:
df['Class'].value_counts() #the dataset has 111 class 0 instances and 97 class 1 instances

0    111
1     97
Name: Class, dtype: int64

In [14]:
df.dropna(inplace=True) #deleting any rows with missing values

In [15]:
df.isnull().any().isnull().any() #confirming there are no missing values in the dataframe

False

In [16]:
X=np.array(df[list(df.columns[:-1])]) 

In [17]:
y=df['Class']

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y, random_state=0) 

In [19]:
logReg.fit(X_train,y_train) #fitting Logistic Regression on Original Dataset

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
logReg_score=logReg.score(X_test,y_test) #Test score
print(logReg_score)

0.75


In [21]:
svc.fit(X_train,y_train) #fitting Support Vector Machines to the original dataset

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
svc_score=svc.score(X_test,y_test)
print(svc_score)

0.826923076923


In [26]:
decClf.fit(X_train,y_train) #fitting decision tree classifer to the original dataset

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [27]:
decClf_score=decClf.score(X_test,y_test)
print(decClf_score)

0.769230769231


Now we will try to run SDSFS algorithm to check if it can improve the accuracy of the classifiers

In [55]:
agents,agentFIndex,agentStatus,agentScores=SDSFS(X,y,estimators,150,120,5,30)

In [56]:
max(agentScores) #one of the agents achieved an accuracy of 86.5 percent

0.87820512820512819

In [57]:
np.argmax(np.array(agentScores)) #agent 29 scored achieved the highest accuracy

36

In [58]:
agents[36].shape #the agent that scored 86.5% accuracy as just 24 features against 60 features in the dataset

(208, 24)

Now we will train the models on this agent instead of original dataset

In [59]:
X_train,X_test,y_train,y_test=train_test_split(agents[36],y, random_state=0) 

In [60]:
logReg.fit(X_train,y_train)

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
logReg_score2=logReg.score(X_test,y_test) #accuracy improved from 75 to 88
print(logReg_score2)

0.884615384615


In [62]:
svc.fit(X_train,y_train)

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [65]:
svc_score2=svc.score(X_test,y_test) #accuracy increased by 2 percent
print(svc_score2)

0.846153846154


In [66]:
decClf.fit(X_train,y_train) 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [67]:
decClf_score2=decClf.score(X_test,y_test) #accuracy increased by 10 percent
print(decClf_score2)

0.865384615385


In [72]:
final_result=pd.DataFrame([[logReg_score,svc_score,decClf_score],[logReg_score2,svc_score2,decClf_score2]], columns=['Logistic Regression','SVM','Decision Tree'], index=['original dataset','SDSFS subset'])

In [75]:
final_result

,Logistic Regression,SVM,Decision Tree
original dataset,0.750000,0.826923,0.769231
SDSFS subset,0.884615,0.846154,0.865385


Accuracy improved for all the models with the subset obtained from SDSFS algorithm. I'm still trying to improve the algorithm to get more consistent and reliable results.